<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/Notebooks/GPTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import time
import random

import pandas as pd
import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from tqdm import tqdm

!pip install transformers

from transformers import (
	AdamW,
	GPT2LMHeadModel,
	GPT2Tokenizer, 
	GPT2Config,
	get_linear_schedule_with_warmup
)

     |████████████████████████████████| 1.9MB 6.3MB/s 
     |████████████████████████████████| 3.2MB 22.0MB/s 
     |████████████████████████████████| 890kB 35.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=257cc2ad040f95fce13a01c16195e4cc4c0eb0ffcc35e31abda011f47240eb1a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!git clone https://github.com/gagan3012/keytotext

Cloning into 'keytotext'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 123 (delta 54), reused 55 (delta 16), pack-reused 0
Receiving objects: 100% (123/123), 2.53 MiB | 11.03 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [3]:
class BOTHData(Dataset):
	def __init__(self, tokenizer, max_source_length,
		max_target_length, type_path):
		super().__init__()
		self.tokenizer = tokenizer
		self.max_source_length = max_source_length
		self.max_target_length = max_target_length
		self.input_ids = []
		self.attn_masks = []
		self._build(type_path)
	
	def __len__(self):
		return len(self.input_ids)
	
	def __getitem__(self, index):

		return self.input_ids[index], self.attn_masks[index]

	def _build(self, type_path):
		if type_path == 'train':
			#df1 = pd.read_csv('/content/keytotext/data/dart_train.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_train.csv')
		elif type_path == 'eval':
			#df = pd.read_csv('/content/keytotext/data/dart_dev.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_dev.csv')
		else:
			#df = pd.read_csv('/content/keytotext/data/dart_test.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_test.csv')
		
	#	df = pd.concat([df1, df2])
		# n = 1 
		# df = df.head(int(len(df)*(n/100)))

		for index, row in df.iterrows():
				line = row['input_text']
				target = row['target_text']
				encodings = self.tokenizer('<|startoftext|>'+ line + ' = ' + target + '<|endoftext|>', truncation=True, max_length=self.max_source_length, padding="max_length")
				self.input_ids.append(torch.tensor(encodings['input_ids']))
				self.attn_masks.append(torch.tensor(encodings['attention_mask']))

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') 

print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [5]:
batch_size = 4
train_dataset = BOTHData(tokenizer, 512, 512, 'train')
val_dataset = BOTHData(tokenizer, 512, 512, 'val')
test_dataset = BOTHData(tokenizer, 512, 512, 'test')

In [6]:
train_dataloader = DataLoader(
			train_dataset,  # The training samples.
			sampler = RandomSampler(train_dataset), # Select batches randomly
			batch_size = batch_size # Trains with this batch size.
		)

validation_dataloader = DataLoader(
			val_dataset, # The validation samples.
			sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
			batch_size = batch_size # Evaluate with this batch size.
		)
test_dataloader = DataLoader(
			test_dataset, # The validation samples.
			sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
			batch_size = batch_size # Evaluate with this batch size.
		)

In [7]:
configuration = GPT2Config.from_pretrained('distilgpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("distilgpt2", config=configuration)

model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [8]:
device = torch.device("cuda:0")
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [9]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [10]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 1000

In [11]:
optimizer = AdamW(model.parameters(),
				  lr = learning_rate,
				  eps = epsilon
				)

In [12]:
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
											num_warmup_steps = warmup_steps, 
											num_training_steps = total_steps)
total_steps

44285

In [13]:
def format_time(elapsed):
	return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [14]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

In [15]:
!pip install wandb

     |████████████████████████████████| 2.0MB 5.2MB/s 
     |████████████████████████████████| 133kB 35.7MB/s 
     |████████████████████████████████| 163kB 31.9MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=d4c6540f80e3a95a307ef71879ea181b8d0ff0a861004c07b3560f66c95e027a
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=2e541095db01d18b6ce13499d80a8877604ae9a5dcd88f9b930f432de4d8b692
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [16]:
%env WANDB_PROJECT = keytotext
import wandb

wandb.init(project="keytotext")

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                    labels=b_labels,
                    attention_mask=b_masks,
                    token_type_ids=None
                    )

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss
        wandb.log({'Train Loss': batch_loss,'Epoch':epoch_i})
# Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss,
                                                                         elapsed))

            model.eval()

            sample_outputs = model.generate(
                                            bos_token_id=random.randint(1, 30000),
                                            do_sample=True,
                                            top_k=50,
                                            max_length=200,
                                            top_p=0.95,
                                            num_return_sequences=1)
            for i, sample_output in enumerate(sample_outputs):
                print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

# Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

# Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

# ========================================
#               Validation
# ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

# Evaluate data for one epoch
    for batch in tqdm(validation_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,
                        #                            token_type_ids=None,
                        attention_mask=b_masks,
                        labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss
        wandb.log({'Eval Loss': batch_loss,'Epoch':epoch_i})

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

# Record all statistics from this epoch.
    training_stats.append(
    {
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss,
        'Training Time': training_time,
        'Validation Time': validation_time
    })


env: WANDB_PROJECT=keytotext


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


0it [00:00, ?it/s]


======== Epoch 1 / 5 ========
Training...


185it [01:34,  1.95it/s]

KeyboardInterrupt: ignored

In [ ]:
!mkdir model
torch.save(model.state_dict(),'model/pytorch_model.bin')